<a href="https://colab.research.google.com/github/harshavardhan9603/Fake-News-Prediction-LogisticRegression-/blob/main/Fake_News_Prediction_ipynb_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [ ]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('Copy of train.csv',encoding = 'unicode_escape',engine='python',on_bad_lines='skip')

In [ ]:
news_dataset.shape

(20822, 5)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id                 0
title            443
author           962
text             619
label            713
                ... 
Unnamed: 681    5425
Unnamed: 682    5425
Unnamed: 683    5425
Unnamed: 684    5425
Unnamed: 685    5425
Length: 686, dtype: int64

In [ ]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

<ipython-input-32-5db6adfd5692>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']


In [ ]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didnât Even...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
5420    shorty U.S. Near Bottom in Public âConfidenc...
5421    Sheryl Gay Stolberg and Erik Eckholm Virginia ...
5422    Nicholas St. Fleur Prey: The Reason Turtles Fi...
5423    TomÃ¡s Fuentes Black Mirror prepara un capÃ­tu...
5424    Neil W. McCabe Exclusive-Navy SEAL Rep. Scott ...
Name: content, Length: 5425, dtype: object


In [ ]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']



In [ ]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didnât Even See Comeyâs...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
5420  4523  U.S. Near Bottom in Public âConfidence in El...   
5421  4524  Virginia Governor Restores Voting Rights to Fe...   
5422  4525  Prey: The Reason Turtles First Came Out of The...   
5423  4526  Black Mirror prepara un capÃ­tulo sobre el dÃ­...   
5424  4527  Exclusive-Navy SEAL Rep. Scott Taylor: Trumpâ...   

                                    author  \
0                            Darrell Lucus   
1                          Daniel J. Flynn   
2                       Consortiumnews.com   
3                          Jessica 

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
5420    shorti u near bottom public confid elect mexic...
5421    sheryl gay stolberg erik eckholm virginia gove...
5422    nichola st fleur prey reason turtl first came ...
5423    tom fuent black mirror prepara un cap tulo sob...
5424    neil w mccabe exclus navi seal rep scott taylo...
Name: content, Length: 5425, dtype: object


In [ ]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'nichola st fleur prey reason turtl first came shell new york time'
 'tom fuent black mirror prepara un cap tulo sobr el en que tu madr se instal whatsapp'
 'neil w mccabe exclus navi seal rep scott taylor trump navi seal widow tribut one power thing ever seen breitbart']


In [ ]:
print(Y)

['1' '0' '1' ... '0' '1' '0']


In [ ]:
Y.shape

(5425,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 8622)	0.28985002551390465
  (0, 7383)	0.24278457448886842
  (0, 4956)	0.3518372208429729
  (0, 4805)	0.3013426746068434
  (0, 4319)	0.24814666613448716
  (0, 3931)	0.22403647985783354
  (0, 2775)	0.22921818883804554
  (0, 2129)	0.2655171237425678
  (0, 2021)	0.3518372208429729
  (0, 1651)	0.2521347361687528
  (0, 1392)	0.3815322142599286
  (0, 167)	0.27332792869223516
  (1, 9163)	0.2952557118893075
  (1, 3832)	0.1905352235144784
  (1, 3102)	0.7197592504797194
  (1, 2004)	0.27013369341775006
  (1, 1564)	0.19274179045257003
  (1, 1262)	0.3795937425705044
  (1, 1080)	0.1579244843018064
  (1, 859)	0.28162715121033227
  (2, 8583)	0.4195451636716658
  (2, 5321)	0.4761533265996163
  (2, 3359)	0.3401064720805493
  (2, 3033)	0.38633341167135443
  (2, 1748)	0.4711367333873954
  :	:
  (5423, 5362)	0.24912228484332372
  (5423, 5005)	0.2701481575202259
  (5423, 4173)	0.24912228484332372
  (5423, 3226)	0.2423534657091044
  (5423, 2633)	0.21579706510549634
  (5423, 2565)	0.2213275930322023
  (5

Splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)


LogisticRegression()

Evaluation

accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.881336405529954


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.8258064516129032


Making a Predictive System

In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['1']
The news is Fake


In [ ]:
print(Y_test[3])